# O output desse processamento deve ser um arquivo com schema pronto e juntando todos os arquivos feitos no passo 1

##1. Importando as bibliotecas

In [2]:
import subprocess
import pandas as pd

##2. Extraindo os arquivos gerados no passo 1

In [8]:
subprocess.run(['unrar','x','/content/drive/MyDrive/tsg.rar'])

CompletedProcess(args=['unrar', 'x', '/content/drive/MyDrive/tsg.rar'], returncode=0)

##3. Criando pastas de apoio

In [9]:
subprocess.run(['mkdir','/content/Passo1'])
subprocess.run(['mkdir','/content/Passo2'])
subprocess.run(['mkdir','/content/Passo3'])

CompletedProcess(args=['mkdir', '/content/Passo3'], returncode=1)

##4. Criando uma lista com o nome dos arquivos

In [10]:
files = subprocess.getoutput(['ls','/content/'])
files = files.split()
files.remove('Passo1')
files.remove('Passo2')
files.remove('Passo3')
files.remove('sample_data')
files.remove('drive')


##5. Transformando os dados numéricos

In [11]:
for fi in files:
  fin = open(fi, "rt")
  fout = open('/content/Passo1/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace(',', '.'))
fin.close()
fout.close()

In [12]:
for fi in files:
  fin = open('/content/Passo1/'+fi, "rt")
  fout = open('/content/Passo2/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace(';', ','))
fin.close()
fout.close()

##6. Cria novos arquivos preparados como variaveis, retirando valores nulos ou errados de datas que vieram ruins e salvando 


In [13]:
for fi in files:  
  df = pd.read_csv('/content/Passo2/'+fi)
  df['Data'] = pd.to_datetime(df['DATE (yyyy-mm-ddThh:mi:ssZ)'],errors='coerce')
  df.dropna(inplace=True)
  df.drop_duplicates(inplace=True)
  df['ANO'] = df['Data'].dt.year
  df['MES'] = df['Data'].dt.month
  df['DIA'] = df['Data'].dt.day
  df['LATITUDE'] = df['LATITUDE (degree_north)']
  df['LONGITUDE'] = df['LONGITUDE (degree_east)']
  df['SALINIDADE'] = df['PSAL LEVEL1 (psu)']
  df['TIPO'] = 'TSG'
  dx=df[['TIPO','ANO','MES','DIA','LATITUDE','LONGITUDE','SALINIDADE']]
  dx.to_csv('/content/Passo3/'+fi)

##7. Junte os arquivos

In [25]:
df2 = pd.DataFrame()
for fi in files:  
  df = pd.read_csv('/content/Passo3/'+fi)
  df2 = pd.concat([df2,df])

##8. Veja se tá tudo ok

In [27]:
df2 

,Unnamed: 0,TIPO,ANO,MES,DIA,LATITUDE,LONGITUDE,SALINIDADE
0,0,TSG,2008,3,12,7.5000,-79.8700,32.320
1,2,TSG,2008,3,12,8.0500,-79.7000,32.150
2,3,TSG,2008,3,12,8.3200,-79.6200,32.200
3,4,TSG,2008,3,12,8.5800,-79.5300,32.310
4,14,TSG,2008,3,14,11.3800,-78.8200,35.800
...,...,...,...,...,...,...,...,...
2255358,2938943,TSG,2017,9,15,35.9213,-75.4346,29.977
2255359,2938944,TSG,2017,9,15,35.9264,-75.4369,30.033
2255360,2938945,TSG,2017,9,15,35.9315,-75.4393,29.955
2255361,2938946,TSG,2017,9,15,35.9365,-75.4420,29.812


##9. Salve o df no drive


In [28]:
df2.to_csv('/content/drive/MyDrive/dados_tsg.csv')